In [39]:
print('Hello LangChain')

Hello LangChain


In [40]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [41]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="자바는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [42]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001EDEF8C4AD0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EDEFB26030> root_client=<openai.OpenAI object at 0x000001EDEFBBA5A0> root_async_client=<openai.AsyncOpenAI object at 0x000001EDEF8C5640> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [43]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("" \
"You are an expert in AI Expert." \
"Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert.Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [44]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### LCEL
* Prompt + LLM + outputparser를 Chain으로 연결하기

In [45]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [46]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 방식과 유사합니다. 컴퓨터가 데이터를 통해 스스로 학습하고, 주어진 문제에 대한 해결 능력을 키우는 것입니다. 여기에는 머신러닝과 딥러닝이 포함됩니다.

1. **데이터 수집**: 우선 인공지능이 학습할 데이터를 수집합니다. 이 데이터는 문제에 대한 답을 포함하고 있어야 합니다.

2. **데이터 전처리**: 수집한 데이터를 깨끗하게 정리하고, 필요한 경우 데이터를 변환하거나 가공합니다.

3. **모델 선택**: 학습에 사용할 알고리즘 또는 모델을 선택합니다. 이 모델은 데이터로부터 패턴을 학습할 수 있는 구조로 되어 있습니다.

4. **학습**: 준비한 데이터를 모델에 입력하여 학습시킵니다. 이 과정에서는 모델이 입력된 데이터로부터 패턴을 인식하고, 그 패턴을 통해 결정을 내릴 수 있도록 합니다. 모델은 예측을 하고, 실제 답과의 오차를 계산합니다. 이 오차를 줄이기 위해 모델의 내부 매개변수(가중치)를 조정하는 과정을 반복합니다.

5. **평가**: 학습이 완료된 후, 모델의 성능을 평가합니다. 이는 별도의 테스트 데이터를 사용하여 모델의 예측 정확도를 확인하는 과정입니다.

6. **튜닝**: 모델의 성능이 만족스럽지 않을 경우, 모델의 구조, 학습 데이터, 하이퍼파라미터(모델의 설정값) 등을 조정하여 성능을 개선합니다.

예를 들어, 이미지 인식 모델을 만든다고 가정해 봅시다. 

- **데이터 수집**: 인터넷에서 고양이, 개 등 다양한 동물의 사진을 수집합니다.
- **데이터 전처리**: 이미지의 크기를 조정하고, 노이즈를 제거하는 등의 작업을 합니다.
- **모델 선택**: 이미지를 분류할 수 있는 신경망 모델을 선택합니다.
- **학습**: 고양이는 1, 개는 0이라는 라벨과 함께 이미지를 모델에 입력하여 학습합니다. 모델은 이미지를 보고 어떤 동물인지 예측하고, 실제 답과 비교하여 오차를 줄입니다.
- **

In [47]:
# chain 호출
try:
    result = chain2.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe같은 products가 있어."})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 AI 관련 제품을 제공하는 회사입니다. LangChain의 주요 제품은 다음과 같습니다:

1. **LangSmith**: LangSmith는 LangChain에서 제공하는 대화형 AI 플랫폼입니다. 개발자가 쉽게 대화형 AI 모델을 구축, 테스트 및 배포할 수 있도록 지원합니다. LangSmith를 통해 개발자는 자연어 처리(NLP) 및 대화형 AI 모델을 빠르게 구축하고, 이를 다양한 애플리케이션에 통합할 수 있습니다.

2. **LangServe**: LangServe는 LangChain에서 제공하는 API 기반의 언어 모델 서비스입니다. LangServe를 통해 개발자는 사전 훈련된 언어 모델을 쉽게 사용할 수 있으며, 이를 자신의 애플리케이션에 통합하여 자연어 처리 기능을 제공할 수 있습니다.

3. **LangChain**: LangChain의 핵심 제품인 LangChain 자체는 개발자가 자연어 처리 및 대화형 AI 모델을 구축하고 배포할 수 있는 프레임워크입니다. LangChain은 다양한 언어 모델과 데이터를 지원하며, 개발자가 쉽게 모델을 훈련시키고 애플리케이션에 통합할 수 있도록 돕습니다.

이러한 제품들은 LangChain이 제공하는 AI 솔루션의 일부이며, 개발자와 기업이 자연어 처리 및 대화형 AI를 활용할 수 있도록 지원합니다. LangChain의 제품들은 사용자의 요구에 따라 다양한 방식으로 활용될 수 있으며, 자연어 처리 분야에서 강력한 도구로 평가받고 있습니다.


### Runnable의 stream() 함수 호출

In [48]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해주세요."})
    # 스트리밍 출력
    print(answer)

    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")

<generator object RunnableSequence.stream at 0x000001EDF24417B0>
인공지능 모델의 학습 원리는 크게 다음과 같은 과정으로 설명할 수 있습니다.

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해서는 먼저大量的의 데이터가 필요합니다. 이 데이터는 문제의 유형에 따라 달라지며, 예를 들어 이미지 인식 모델을 만들려면 이미지 데이터를 수집하고, 자연어 처리 모델을 만들려면 텍스트 데이터를 수집합니다.

2. **데이터 전처리**: 수집된 데이터는 모델에 사용하기 전에 전처리 과정을 거칩니다. 이 과정에서는 데이터의 오류를 수정하거나, 데이터를 정규화하거나, 필요한 경우 데이터를 변환하는 작업 등이 포함됩니다.

3. **모델 선택**: 인공지능 모델에는 여러 가지 유형이 있으며, 해결하려는 문제의 성격에 따라 적합한 모델을 선택해야 합니다. 예를 들어, 이미지 인식 문제는 합성곱 신경망(CNN), 자연어 처리 문제는 순환 신경망(RNN) 또는 트랜스포머 모델을 사용할 수 있습니다.

4. **모델 학습**: 선택된 모델에 전처리된 데이터를 입력하여 모델을 학습시킵니다. 이 과정에서는 모델이 데이터로부터 패턴이나 규칙을 스스로 발견할 수 있도록 합니다. 모델의 학습에는 크게 **감독 학습**과 **비감독 학습** 두 가지 방법이 있습니다.
   - **감독 학습**: 모델이 입력 데이터와 그에 대응하는 출력(레이블)을 함께 학습하는 방법입니다. 모델은 입력 데이터와 출력 간의 관계를 학습하여, 새로운 입력 데이터에 대해 올바른 출력을 예측할 수 있습니다.
   - **비감독 학습**: 모델이 입력 데이터만으로 학습하는 방법입니다. 이 경우, 모델은 데이터에서 패턴이나 구조를 발견하려고 시도하며, 주로 데이터의 클러스터링이나 차원 축소에 사용됩니다.

5. **모델 평가**: 학습된 모델의 성능을 평가합니다. 이를 위해 학습 데이터와는 별도의 테스트 데이터를 사용하여 모델의 성능을 측정합니다. 평가 지표는 문제

### Multi Chain
* 첫 번째 Chain의 출력이, 두 번째 Chain의 입력이 된다.
* 두 개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [49]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()


client=<openai.resources.chat.completions.completions.Completions object at 0x000001EDEF8C7F20> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EDEF8C6ED0> root_client=<openai.OpenAI object at 0x000001EDEF8C53A0> root_async_client=<openai.AsyncOpenAI object at 0x000001EDEDA76030> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [50]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "Drama"})
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

The movie I recommend is **'뷰티풀 마인드'**.

Here is a summary of the movie in three sentences:

The film tells the story of John Nash, a brilliant mathematician who struggles with schizophrenia while studying at Princeton University. As Nash's mental health deteriorates, he faces numerous challenges and setbacks, but ultimately finds love, friendship, and a way to overcome his illness. Through its powerful portrayal of Nash's journey, the movie aims to break down stigmas surrounding mental illness and highlight the importance of human connection and resilience.


### PromptTemplate 여러개 연결하기

In [51]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

ChatGPT 모델의 학습 원리는 다음과 같습니다.

ChatGPT는 대규모 언어 모델로, 수십억 개의 매개변수를 가지고 있습니다. 이 모델은 인터넷에서 수집한 대규모 텍스트 데이터를 기반으로 학습하여 자연어 처리 능력을 개발합니다. 학습 과정에서 모델은 입력된 텍스트에 대한 다음 단어를 예측하도록 훈련되며, 이를 통해 대화와 같은 자연어 상호작용을 생성할 수 있습니다.

ChatGPT 모델의 장점은 다음과 같습니다.

* 자연어 이해 및 생성 능력
* 대규모 지식 데이터베이스에 기반한 답변 제공
* 다양한 주제 및 상황에 대한 대화 지원
* 높은 수준의 언어 이해 및 생성 능력

ChatGPT 모델과 비슷한 AI 모델은 다음과 있습니다.

* LLaMA
* PaLM
* BERT
* RoBERTa


In [52]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 요약해서 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 요약해서 한국어로 답변해 주세요.', 'llama-4 모델의 학습 원리를 4 문장으로 요약해서 한국어로 답변해 주세요.']


In [53]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) # AI Message 타입
    print(response)

content='GPT-4 모델은 대규모 데이터셋을 기반으로 하는 머신러닝 알고리즘을 사용하여 학습되며, 자연어 처리 작업에 특화된 트랜스포머 아키텍처를 활용합니다. 이 모델은 주어진 문맥을 이해하고 적절한 응답을 생성하도록 학습되며, 강화 학습 및 지도 학습을 통해 지속적으로 개선됩니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 30, 'total_tokens': 101, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.212888058, 'prompt_time': 0.002649579, 'completion_time': 0.140206777, 'total_time': 0.142856356}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'id': 'chatcmpl-852b220a-9389-4d98-8062-3ee97fd5c66a', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--b52a3097-2d06-410f-b3e6-64724642391c-0' usage_metadata={'input_tokens': 30, 'output_tokens': 71, 'total_tokens': 101, 'input_token_details': {}, 'output_token_details': {}}
content='Gemma는 컴퓨터가 자연어 처리를 수행하도록 특별히 설계된 인공지능(AI) 언어 모델입니다. 대규모 텍스트 데이터 세트에 대해 학습되었으며, 이 학습을 통해 주어진 맥락에서 다음에 오는 문장이

### ChatMessagePromptTemplate
* SystemMessagePrompt, HumanMessagePromptTemplate, AIMessagePromptTemplate를 사용

In [54]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가입니다. 명확하고 자세하게 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "{topic}에 대한 예시 답입니다."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요??")

# LLM 호출
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 

## 딥러닝의 정의

딥러닝은 인공신경망을 기반으로 하는 머신러닝의 한 분야입니다. 인공신경망은 인간의 뇌를 모방한 구조로, 데이터를 처리하고 학습하는 방식입니다. 딥러닝은 이러한 인공신경망을 깊게 쌓아 올림으로써 복잡한 데이터와 패턴을 학습하고 예측할 수 있습니다.

## 딥러닝의 작동 원리

딥러닝은 다음과 같은 단계를 거쳐 작동합니다.

1. **데이터 수집**: 딥러닝 모델은 대량의 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 오디오 등 다양한 형태일 수 있습니다.
2. **데이터 전처리**: 수집된 데이터는 전처리 과정을 거칩니다. 이 과정에서는 데이터의 잡음 제거, 정규화, 변환 등이 수행됩니다.
3. **모델 설계**: 딥러닝 모델은 인공신경망을 기반으로 설계됩니다. 모델은 입력층, 은닉층, 출력층으로 구성되며, 각 층은 여러 개의 노드로 구성됩니다.
4. **학습**: 모델은 학습 데이터를 사용하여 학습합니다. 학습 과정에서는 모델의 가중치가 조정되며, 모델의 성능이 최적화됩니다.
5. **평가**: 학습된 모델은 평가 데이터를 사용하여 평가됩니다. 평가 과정에서는 모델의 성능이 측정되며, 모델의 정확도가 확인됩니다.

## 딥러닝의 특징

딥러닝은 다음과 같은 특징을 가지고 있습니다.

* **자동화된 특징 학습**: 딥러닝 모델은 데이터로부터 자동으로 특징을 학습할 수 있습니다. 이는 기존의 머신러닝 방법에서는 어려웠던 작업입니다.
* **대량의 데이터 요구**: 딥러닝 모델은 대량의 데이터를 필요로 합니다. 이는 모델의 성능을 향상시키기 위해 필요합니다.
* **고성능 컴퓨팅 요구**: 딥러닝 모델은 고성능 컴퓨팅을 필요로 합니다. 이는 모델의 학습과 평가를 빠르게 수행하기 위해 필요합니다.

## 딥러닝의 응용 분야

딥러닝은 다양한 분야에서 응용되고 있습니다. 예를 들어,

* **이미지 인식**: 딥러닝은 이미지 인식 분야에서 널리 사용됩니다. 예를 들어, 얼굴 인식, 객체 인식, 이미지 분류 등이 있습니다.
* **자

### FewShotPromptTemplate
* 예시를 제공하는 프롬프트

In [60]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 태양과 가장 가까운 행성으로, 매우 작고 온도가 극심하게 변합니다.
2. **금성**: 두꺼운 대기로 인해 극심한 온실 효과가 있습니다.
3. **지구**: 생명체가 존재하는 유일한 행성입니다.
4. **화성**: 붉은색이며, 과거에는 물이 존재했을 것으로 추정됩니다.
5. **목성**: 태양계에서 가장 큰 행성으로, 가스 행성입니다.
6. **토성**: 아름다운 고리를 가지고 있는 가스 행성입니다.
7. **천왕성**: 자전축이 기울어져 있어 극단적인 계절 변화를 겪습니다.
8. **해왕성**: 태양계에서 가장 먼 행성으로, 강한 바람이 불고 있습니다.


### PartialPromptTemplate
* 프롬프트 입력 값에 함수 호출이나 외부 API를 호출한 동적인 값을 대입할 수 있음

In [61]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month
    
    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"



# 프롬프트 템플릿 정의 (부분 변수 적용)
# prompt = PromptTemplate(
#     template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
#     input_variables=["phenomenon"],  # 사용자 입력 필요
#     partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
# )

season = get_current_season("south")
prompt = ChatPromptTemplate.from_template(
    "{season}에 일어나는 대표적인 지구과학 현상 3가지를 알려주세요."
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 초기화
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = (
    {"season": lambda x: season} |
    prompt |
    model |
    StrOutputParser()

)
# 특정 계절의 현상 질의
result = chain.invoke({})


# 결과 출력
print(f"🔹 프롬프트: {prompt}")
print(f"🔹 모델 응답: {season}에 발생하는 자연 현상 :\n {result}")

🔹 프롬프트: input_variables=['season'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['season'], input_types={}, partial_variables={}, template='{season}에 일어나는 대표적인 지구과학 현상 3가지를 알려주세요.각 현상에 대해 간단한 설명을 포함해주세요.'), additional_kwargs={})]
🔹 모델 응답: 겨울에 발생하는 자연 현상 :
 1.  **북극광(오로라)**: 북극광은 태양의 태양풍이 지구의 자기장에 닿아 발생합니다. 태양풍의 전하 입자가 지구의 자기장에 의해 방향을 바꾸어 대기권과 충돌하면서 발생되는 빛입니다. 북극광은 주로 북극 지역에서 볼 수 있으며, 태양풍이 강할수록 더 밝고 화려하게 나타납니다.

2.  **빙하 형성**: 겨울이 되면 기온이 낮아지면서 눈과 비가 얼어붙어 빙하가 형성됩니다. 빙하는 지구의 극지방에서 발견되며, 빙하의 형성 과정은 수천 년에 걸쳐 일어나고 있습니다. 빙하는 지구의 기후 변화에 민감하게 반응하기 때문에 기후 연구에 중요한 지표입니다.

3.  **극야 현상**: 극야 현상은 북극과 남극 지역에서 겨울에 일어나는 현상으로, 태양이 하루 종일 지평선 아래에 머물러 있어 낮이 전혀 없는 현상을 말합니다. 이 기간 동안에는 태양이 뜨지 않기 때문에 극지방에서는 밤만 지속됩니다. 극야 현상은 북극과 남극에서 각각 6개월씩 발생하며, 이 기간 동안에는 밤이 지속됩니다.


In [59]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# {info} 변수에 API에서 받은 환율 정보를 동적으로 반영
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제 미치는 영향 및 향후에 환율예상값에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

print(prompt)

# LLM 모델 설정 (GPT-4o-mini 사용)
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# 결과 출력
print("🔹 프롬프트:", prompt.format())
print("🔹 모델 응답:", response.content)

input_variables=[] input_types={} partial_variables={'info': '1달러 = 1359.26원'} template='현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제 미치는 영향 및 향후에 환율예상값에 대한 분석을 제공해 주세요.'
🔹 프롬프트: 현재 1달러 = 1359.26원 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제 미치는 영향 및 향후에 환율예상값에 대한 분석을 제공해 주세요.
🔹 모델 응답: 현재 환율(1달러 = 1359.26원)은 한국 경제에 여러 가지 영향을 미치고 있습니다. 우선, 높은 환율은 수출 중심의 한국 경제에 긍정적인 영향을 줄 수 있습니다. 

1. **수출 증가**: 높은 환율은 한국의 수출 상품 가격을 낮추어 해외 시장에서 경쟁력을 높일 수 있습니다. 이는 자동차, 반도체, 철강 등 주요 수출 품목의 가격 경쟁력을 강화하여 수출량을 증가시킬 수 있습니다. 수출 증가는 무역 수지 개선과 외화 수입 증가로 이어질 수 있습니다.

2. **물가 상승**: 반면, 높은 환율은 수입 물가를 상승시켜 국내 물가 상승 압력을 가중시킬 수 있습니다. 원자재, 에너지, 식품 등 수입 의존도가 높은 품목들의 가격이 상승하면, 이는 곧 국내 소비자 물가 상승으로 이어질 수 있습니다. 이는 가계의 구매력을 감소시키고, 전반적인 경제 불안을 야기할 수 있습니다.

3. **기업 수익성**: 높은 환율은 수출 비중이 큰 기업들의 수익성을 개선할 수 있지만, 수입 원자재 가격 상승으로 인해 기업들의 비용 부담이 증가할 수도 있습니다. 특히, 수입 원자재에 의존도가 높은 중소기업이나 특정 산업에서는 비용 증가로 인한 수익성 악화가 우려됩니다.

4. **금융 시장**: 외환 시장에서 원화 가치가 낮아지면, 외국인 투자자의 국내 주식 및 채권 시장 투자에 영향을 줄 수 있습니다. 원화 가치 하락은 외국인에게 국내 자산의 매력을 감소시킬 수 있어, 자본 유출입에 